In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import matplotlib.pyplot as plt

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler


import folium # map rendering library


from geopy.geocoders import Nominatim
from geopandas import gpd

print('Libraries imported.')

Libraries imported.


## Data Import

In [2]:
# Import of Prague Cadastral districts data and superiro Administrtive districts
prague = pd.read_csv("PRG.csv",encoding='windows-1250', sep = ";")
prague.head()

,lon,lat,name,name_AD
0,14.418190,50.089807,Josefov,Praha 1
1,14.421599,50.086638,Staré Město,Praha 1
2,14.404111,50.086318,Malá Strana,Praha 1
3,14.392468,50.088993,Hradčany,Praha 1
4,14.425256,50.079334,Nové Město,Praha 1


In [3]:
# Import of Bratislava Cadastral districts data and superiror Administrative districts
bratislava = pd.read_csv("BA.csv",encoding='windows-1250', sep = ";")
bratislava.head()

,lon,lat,NAZOV_UTJ,NAZOV_OKR
0,17.099373,48.151005,Stare Mesto,Bratislava I
1,17.208425,48.128408,Podunajske Biskupice,Bratislava II
2,17.144011,48.141358,Nivy,Bratislava II
3,17.170745,48.169519,Trnavka,Bratislava II
4,17.167616,48.150173,Ruzinov,Bratislava II


In [4]:
# rename columns into same format as Prague data
bratislava = bratislava.rename(columns={'NAZOV_UTJ': 'name', 'NAZOV_OKR':'name_AD'})
bratislava.head()

,lon,lat,name,name_AD
0,17.099373,48.151005,Stare Mesto,Bratislava I
1,17.208425,48.128408,Podunajske Biskupice,Bratislava II
2,17.144011,48.141358,Nivy,Bratislava II
3,17.170745,48.169519,Trnavka,Bratislava II
4,17.167616,48.150173,Ruzinov,Bratislava II


## Visualization of Prague and Bratislava

In [5]:
address_cz = 'Prague'

geolocator_prg = Nominatim(user_agent="Prague_explorer")
location_prg = geolocator_prg.geocode(address_cz)
latitude_prg = location_prg.latitude
longitude_prg = location_prg.longitude
print('The geograpical coordinate of Prague are {}, {}.'.format(latitude_prg, longitude_prg))

The geograpical coordinate of Prague are 50.0874654, 14.4212535.


In [6]:
address_sk = 'Bratislava'

geolocator_sk = Nominatim(user_agent="Bratislava_explorer")
location_sk = geolocator_sk.geocode(address_sk)
latitude_sk = location_sk.latitude
longitude_sk = location_sk.longitude
print('The geograpical coordinate of Bratislava are {}, {}.'.format(latitude_sk, longitude_sk))

The geograpical coordinate of Bratislava are 48.1516988, 17.1093063.


In [7]:
# Map of Prague
prague_map = folium.Map(location = [latitude_prg, longitude_prg], zoom_start=10)

# add markers to map
for lat, lng, name_AD, name in zip(prague['lat'], prague['lon'], prague['name'], prague['name_AD']):
    label = '{}, {}'.format(name, name_AD)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(prague_map) 
    
prague_map

In [8]:
# Map of Bratislava
ba_map = folium.Map(location = [latitude_sk, longitude_sk], zoom_start=10)

# add markers to map
for lat, lng, name_AD, name in zip(bratislava['lat'], bratislava['lon'], bratislava['name'], bratislava['name_AD']):
    label = '{}, {}'.format(name, name_AD)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ba_map) 
    
ba_map

## Try one cadastral in Prague

In [9]:
CLIENT_ID = 'MERBJETXKZXCJKKZTRZJF4JU2J2B2FP0SFJOQEQS2XJL2SS3' # your Foursquare ID
CLIENT_SECRET = '3ROUQ4TRAN5TVVY05UBP13MTZ41I4UXDP3VZPOZYSVKLK5QL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MERBJETXKZXCJKKZTRZJF4JU2J2B2FP0SFJOQEQS2XJL2SS3
CLIENT_SECRET:3ROUQ4TRAN5TVVY05UBP13MTZ41I4UXDP3VZPOZYSVKLK5QL


In [10]:
## Define own link for API usage
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# Using of function to get all venues for all Prague's cadastral district


In [11]:
def getNearbyVenues(name_cd, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(name_cd, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results if v['venue']['categories'][0]['name'] != 'Bus Stop']) ## I cut off the "bus stop" category.

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['cadastral_name', 
                  'cadastral lat', 
                  'cadastral lon', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
prague_venues = getNearbyVenues(name_cd = prague['name'],
                                latitudes =prague['lat'],
                                longitudes =prague['lon']
)

Josefov
Staré Město
Malá Strana
Hradčany
Nové Město
Vyšehrad
Žižkov
Krč
Michle
Hodkovičky
Braník
Podolí
Nusle
Kunratice
Radlice
Jinonice
Košíře
Hlubočepy
Motol
Smíchov
Holyně
Slivenec
Dejvice
Střešovice
Veleslavín
Vokovice
Břevnov
Ruzyně
Liboc
Nebušice
Lysolaje
Suchdol
Sedlec
Přední Kopanina
Bubeneč
Holešovice
Troja
Březiněves
Čimice
Kobylisy
Bohnice
Dolní Chabry
Ďáblice
Libeň
Karlín
Střížkov
Vysočany
Prosek
Hrdlořezy
Vinohrady
Strašnice
Záběhlice
Vršovice
Malešice
Křeslice
Chodov
Háje
Šeberov
Újezd u Průhonic
Cholupice
Točná
Písnice
Lhotka
Libuš
Kamýk
Komořany
Modřany
Řeporyje
Zadní Kopanina
Stodůlky
Třebonice
Dolní Počernice
Kyje
Hloubětín
Černý Most
Hostavice
Dubeč
Hostivař
Štěrboholy
Dolní Měcholupy
Horní Měcholupy
Petrovice
Kbely
Satalice
Vinoř
Horní Počernice
Běchovice
Klánovice
Koloděje
Újezd nad Lesy
Benice
Kolovraty
Lipany
Královice
Nedvězí u Říčan
Hájek u Uhříněvsi
Pitkovice
Uhříněves
Lipence
Lochkov
Malá Chuchle
Velká Chuchle
Lahovice
Radotín
Zbraslav
Řepy
Sobín
Zličín
Letňa

In [13]:
# Group of prague venues by cadastral_name and sorting
prague_venues.groupby("cadastral_name").count().sort_values(by = ["Venue"], ascending = False).head(20)

,cadastral lat,cadastral lon,Venue,Venue Latitude,Venue Longitude,Venue Category
cadastral_name,,,,,,
Holešovice,100,100,100,100,100,100
Vinohrady,100,100,100,100,100,100
Smíchov,100,100,100,100,100,100
Nové Město,100,100,100,100,100,100
Josefov,100,100,100,100,100,100
Malá Strana,97,97,97,97,97,97
Karlín,66,66,66,66,66,66
Vyšehrad,60,60,60,60,60,60
Staré Město,58,58,58,58,58,58


In [14]:
print('There are {} uniques categories.'.format(len(prague_venues['Venue Category'].unique())))

There are 285 uniques categories.


## Analyze each cadastral district in Prague 

In [15]:
# one hot encoding
prague_onehot = pd.get_dummies(prague_venues[['Venue Category']], prefix="", prefix_sep="")

# add cadastral name column back to dataframe
prague_onehot['cadastral_name'] = prague_venues['cadastral_name'] 

# move last column to the first place
fixed_columns = [prague_onehot.columns[-1]] + list(prague_onehot.columns[:-1])
prague_onehot = prague_onehot[fixed_columns]

prague_onehot.head()

,cadastral_name,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bathing Area,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Casino,Castle,Caucasian Restaurant,Cave,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gelato Shop,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Laser Tag,Library,Lighting Store,Liquor Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Physical Therapist,Pizza Place,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Reservoir,Resort,Restaurant,River,Road,Roof Deck,Rooftop Bar,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel Agency,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo
0,Josefov,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Join back administrative district

In [16]:
# Bacause I want to group results by superior administartive area. I am joining back administrative districts.
prague_venues = prague_onehot.merge(prague['name_AD'],how = "inner", left_on = "cadastral_name", right_on =prague["name"])

# Move last columns to the first position
fixed_columns = [prague_venues.columns[-1]] + list(prague_venues.columns[:-1])
prague_venues = prague_venues[fixed_columns]

prague_venues.head()

,name_AD,cadastral_name,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bathing Area,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Casino,Castle,Caucasian Restaurant,Cave,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gelato Shop,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Laser Tag,Library,Lighting Store,Liquor Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Physical Therapist,Pizza Place,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Reservoir,Resort,Restaurant,River,Road,Roof Deck,Rooftop Bar,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel Agency,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo
0,Praha 1,Josefov,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [17]:
# grouping over name of administrative district
prague_grouped = prague_venues.groupby('name_AD').mean().reset_index()
prague_grouped.head()

,name_AD,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bathing Area,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Casino,Castle,Caucasian Restaurant,Cave,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Czech Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gelato Shop,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Laser Tag,Library,Lighting Store,Liquor Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Physical Therapist,Pizza Place,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Reservoir,Resort,Restaurant,River,Road,Roof Deck,Rooftop Bar,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Travel Agency,Tree,Tunnel,Turkish Restaurant,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo
0,Praha 1,0.0,0.002558,0.0,0.017903,0.01023,0.005115,0.005115,0.002558,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.007673,0.015345,0.

In [18]:
prague_grouped.shape

(22, 286)

# Putting that into a pandas dataframe
#### Function, which sorts the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name_AD']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
prague_venues_sorted = pd.DataFrame(columns=columns)
prague_venues_sorted['name_AD'] = prague_grouped['name_AD']

for ind in np.arange(prague_grouped.shape[0]):
    prague_venues_sorted.iloc[ind, 1:] = return_most_common_venues(prague_grouped.iloc[ind, :], num_top_venues)

prague_venues_sorted.head()

,name_AD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Praha 1,Café,Hotel,Restaurant,Cocktail Bar,Plaza,Italian Restaurant,Czech Restaurant,Coffee Shop,Boutique,Pub
1,Praha 10,Café,Pub,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bistro,Bar,Coffee Shop,Hotel,Vietnamese Restaurant
2,Praha 11,Restaurant,Athletics & Sports,Pharmacy,Supermarket,Electronics Store,Drugstore,Pizza Place,Asian Restaurant,Pet Store,Soccer Field
3,Praha 12,Restaurant,Pizza Place,Asian Restaurant,Park,Gym,Dessert Shop,Pharmacy,Electronics Store,Café,Soccer Field
4,Praha 13,Pub,Pizza Place,Restaurant,Café,Brewery,Reservoir,Plaza,Tennis Court,Beer Garden,Gym / Fitness Center


In [21]:
# set number of clusters. Elbowing method did not say any better distribution.
kclusters = 5

prague_grouped_clustering = prague_grouped.drop('name_AD', 1)
prague_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(prague_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 2, 0, 0, 0, 2, 3, 0])

In [22]:
# Inserting cluster labels into dataframe
prague_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
prague_venues_sorted.head()

,Cluster Labels,name_AD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Praha 1,Café,Hotel,Restaurant,Cocktail Bar,Plaza,Italian Restaurant,Czech Restaurant,Coffee Shop,Boutique,Pub
1,0,Praha 10,Café,Pub,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bistro,Bar,Coffee Shop,Hotel,Vietnamese Restaurant
2,2,Praha 11,Restaurant,Athletics & Sports,Pharmacy,Supermarket,Electronics Store,Drugstore,Pizza Place,Asian Restaurant,Pet Store,Soccer Field
3,2,Praha 12,Restaurant,Pizza Place,Asian Restaurant,Park,Gym,Dessert Shop,Pharmacy,Electronics Store,Café,Soccer Field
4,0,Praha 13,Pub,Pizza Place,Restaurant,Café,Brewery,Reservoir,Plaza,Tennis Court,Beer Garden,Gym / Fitness Center


In [23]:
# Aaverage coordinates for prague administartive districts
prague_final = prague.groupby('name_AD').mean().reset_index()
prague_final.head()

,name_AD,lon,lat
0,Praha 1,14.412325,50.086218
1,Praha 10,14.480233,50.070553
2,Praha 11,14.529319,50.023742
3,Praha 12,14.438577,49.998298
4,Praha 13,14.309058,50.031775


#### Joining lat lon attributes into final table

In [24]:
prague_venues_merged = prague_venues_sorted.merge(prague_final[['lat', 'lon']],how = "inner", left_on = "name_AD", right_on =prague_final["name_AD"])
prague_venues_merged.head()

,Cluster Labels,name_AD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,lat,lon
0,0,Praha 1,Café,Hotel,Restaurant,Cocktail Bar,Plaza,Italian Restaurant,Czech Restaurant,Coffee Shop,Boutique,Pub,50.086218,14.412325
1,0,Praha 10,Café,Pub,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bistro,Bar,Coffee Shop,Hotel,Vietnamese Restaurant,50.070553,14.480233
2,2,Praha 11,Restaurant,Athletics & Sports,Pharmacy,Supermarket,Electronics Store,Drugstore,Pizza Place,Asian Restaurant,Pet Store,Soccer Field,50.023742,14.529319
3,2,Praha 12,Restaurant,Pizza Place,Asian Restaurant,Park,Gym,Dessert Shop,Pharmacy,Electronics Store,Café,Soccer Field,49.998298,14.438577
4,0,Praha 13,Pub,Pizza Place,Restaurant,Café,Brewery,Reservoir,Plaza,Tennis Court,Beer Garden,Gym / Fitness Center,50.031775,14.309058


In [25]:
# Creating variable with own geojason of Prague Administartive Districs
prague_geo = r'administrative_districts_prague - Copy.geojson' ## my local geojason of prague Administrtive Districs
prague_geo

'administrative_districts_prague - Copy.geojson'

In [26]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [27]:
# Visualization of Administrative districts
# I put into visualization the location points as well... There are there maily beacause of popup funciton.

map_clusters_prg_ = folium.Map(location = [latitude_prg, longitude_prg], zoom_start=10)
map_clusters_prg_.choropleth(
    geo_data = prague_geo,
    data = prague_venues_merged,
    columns = ['name_AD','Cluster Labels'],
    key_on='feature.properties.NAZEV_SO',
    fill_color='PuBuGn',
    fill_opacity=0.7, 
    legend_name=" Number of cluster"
)


markers_colors = []
for lat, lon, poi, cluster in zip(prague_venues_merged['lat'], prague_venues_merged['lon'], prague_venues_merged['name_AD'], prague_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_clusters_prg_)
    
map_clusters_prg_

In [28]:
# Checking of Administrative districts - Just change the number
prague_venues_merged.loc[prague_venues_merged['Cluster Labels'] == 0, prague_venues_merged.columns[[1] + list(range(2, prague_venues_merged.shape[1]))]]

,name_AD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,lat,lon
0,Praha 1,Café,Hotel,Restaurant,Cocktail Bar,Plaza,Italian Restaurant,Czech Restaurant,Coffee Shop,Boutique,Pub,50.086218,14.412325
1,Praha 10,Café,Pub,Wine Bar,Cocktail Bar,Vegetarian / Vegan Restaurant,Bistro,Bar,Coffee Shop,Hotel,Vietnamese Restaurant,50.070553,14.480233
4,Praha 13,Pub,Pizza Place,Restaurant,Café,Brewery,Reservoir,Plaza,Tennis Court,Beer Garden,Gym / Fitness Center,50.031775,14.309058
5,Praha 14,Park,Restaurant,Trail,Pub,Drugstore,Electronics Store,Gastropub,Grocery Store,Supermarket,Café,50.096628,14.561325
6,Praha 15,Park,Pizza Place,Pub,Czech Restaurant,Grocery Store,Restaurant,Bookstore,Café,Athletics & Sports,Brewery,50.054868,14.558501
9,Praha 18,Café,Vietnamese Restaurant,Soccer Field,Grocery Store,Pub,Restaurant,Wine Shop,Gym Pool,Bus Station,Dessert Shop,50.152674,14.524368
11,Praha 2,Café,Hotel,Park,Pub,Beer Garden,Historic Site,Playground,Wine Bar,Czech Restaurant,Italian Restaurant,50.065145,14.418725
14,Praha 22,Czech Restaurant,Restaurant,Reservoir,Pub,Trail,Food & Drink Shop,Café,Playground,Campground,Motel,50.023548,14.619234
15,Praha 3,Vietnamese Restaurant,Bar,Café,Restaurant,Steakhouse,Antique Shop,Czech Restaurant,Gymnastics Gym,Gym / Fitness Center,Gym,50.087127,14.469357
16,Praha 4,Restaurant,Park,Pub,Café,Gastropub,Hotel,Bar,Bistro,Chinese Restaurant,Czech Restaurant,50.039395,14.439106


# Using of function to get all venues for all Bratislava's cadastral district


In [29]:
bratislava_venues = getNearbyVenues(name_cd = bratislava['name'],
                                latitudes = bratislava['lat'],
                                longitudes = bratislava['lon']
)

Stare Mesto
Podunajske Biskupice
Nivy
Trnavka
Ruzinov
Vrakuna
Vinohrady
Nove Mesto
Raca
Vajnory
Devinska Nova Ves
Dubravka
Karlova Ves
Devin
Lamac
Zahorska Bystrica
Cunovo
Jarovce
Petrzalka
Rusovce


In [30]:
# Group of prague venues by cadastral_name and sorting. Obviously Bratislava has very less foursquere data
bratislava_venues.groupby("cadastral_name").count().sort_values(by = ["Venue"], ascending = False).head(20)

,cadastral lat,cadastral lon,Venue,Venue Latitude,Venue Longitude,Venue Category
cadastral_name,,,,,,
Petrzalka,16,16,16,16,16,16
Devin,16,16,16,16,16,16
Ruzinov,14,14,14,14,14,14
Devinska Nova Ves,13,13,13,13,13,13
Rusovce,13,13,13,13,13,13
Trnavka,13,13,13,13,13,13
Dubravka,12,12,12,12,12,12
Nove Mesto,11,11,11,11,11,11
Zahorska Bystrica,10,10,10,10,10,10


In [31]:
print('There are {} uniques categories.'.format(len(bratislava_venues['Venue Category'].unique())))

There are 77 uniques categories.


In [32]:
#one hot encoding
bratislava_onehot = pd.get_dummies(bratislava_venues[['Venue Category']], prefix="", prefix_sep="")

# add cadastral name column back to dataframe
bratislava_onehot['cadastral_name'] = bratislava_venues['cadastral_name'] 

# move last column to the first place
fixed_columns = [bratislava_onehot.columns[-1]] + list(bratislava_onehot.columns[:-1])
bratislava_onehot = bratislava_onehot[fixed_columns]

bratislava_onehot.head()

,cadastral_name,Asian Restaurant,Athletics & Sports,BBQ Joint,Bar,Baseball Field,Baseball Stadium,Beer Garden,Bistro,Boarding House,Bookstore,Bowling Alley,Brewery,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Campground,Castle,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Fast Food Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hotel,Hungarian Restaurant,Ice Cream Shop,Italian Restaurant,Lighthouse,Martial Arts School,Massage Studio,Mexican Restaurant,Monument / Landmark,Mountain,Movie Theater,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Restaurant,River,Salad Place,Sandwich Place,Skating Rink,Ski Chairlift,Slovak Restaurant,Snack Place,Soccer Field,Sporting Goods Shop,Supermarket,Tapas Restaurant,Tennis Court,Track,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Stare Mesto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Stare Mesto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Stare Mesto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Stare Mesto,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Stare Mesto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
# grouping over name of cadastral names
bratislava_grouped = bratislava_onehot.groupby('cadastral_name').mean().reset_index()
bratislava_grouped.head()

,cadastral_name,Asian Restaurant,Athletics & Sports,BBQ Joint,Bar,Baseball Field,Baseball Stadium,Beer Garden,Bistro,Boarding House,Bookstore,Bowling Alley,Brewery,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Campground,Castle,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Fast Food Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hotel,Hungarian Restaurant,Ice Cream Shop,Italian Restaurant,Lighthouse,Martial Arts School,Massage Studio,Mexican Restaurant,Monument / Landmark,Mountain,Movie Theater,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Restaurant,River,Salad Place,Sandwich Place,Skating Rink,Ski Chairlift,Slovak Restaurant,Snack Place,Soccer Field,Sporting Goods Shop,Supermarket,Tapas Restaurant,Tennis Court,Track,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Cunovo,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.25,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.250000,0.0,0.0,0.0,0.0,0.250,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Devin,0.0,0.0,0.062500,0.000000,0.000000,0.0,0.000000,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.125000,0.0,0.0625,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.062500,0.0,0.0,0.0,0.062500,0.0,0.000000,0.000000,0.0,0.0625,0.0625,0.00,0.000000,0.000000,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.062500,0.000000,0.0,0.0,0.0,0.0,0.125,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Devinska Nova Ves,0.0,0.0,0.076923,0.153846,0.076923,0.0,0.230769,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.230769,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.076923,0.0,0.0000,0.0000,0.00,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.076923,0.000000,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Dubravka,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.0,0.0,0.083333,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.00,0.083333,0.083333,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.250,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Jarovce,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.0,0.0,0.0,0.166667,0.0,0.333333,0.000000,0.0,0.0000,0.0000,0.00,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['cadastral_name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
bratislava_venues_sorted = pd.DataFrame(columns=columns)
bratislava_venues_sorted['cadastral_name'] = bratislava_grouped['cadastral_name']

for ind in np.arange(bratislava_grouped.shape[0]):
    bratislava_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bratislava_grouped.iloc[ind, :], num_top_venues)

bratislava_venues_sorted.head()

,cadastral_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cunovo,Hungarian Restaurant,Playground,Restaurant,Eastern European Restaurant,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop
1,Devin,Café,Restaurant,Castle,Bistro,Park,Cafeteria,River,Lighthouse,Eastern European Restaurant,Gastropub
2,Devinska Nova Ves,Beer Garden,Café,Bar,Gym,BBQ Joint,Baseball Field,Supermarket,Pizza Place,Yoga Studio,Dog Run
3,Dubravka,Restaurant,Dessert Shop,Ice Cream Shop,Italian Restaurant,Gastropub,Supermarket,Playground,Eastern European Restaurant,Czech Restaurant,Diner
4,Jarovce,Grocery Store,Eastern European Restaurant,Pizza Place,Gastropub,Soccer Field,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant


In [35]:
# set number of clusters
kclusters = 5

bratislava_grouped_clustering = bratislava_grouped.drop('cadastral_name', 1)
bratislava_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bratislava_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 2, 3, 1, 2, 2, 1, 2, 1])

In [36]:
# Inserting numbers for clusters into dataframe
bratislava_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
bratislava_venues_sorted.head()

,Cluster Labels,cadastral_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Cunovo,Hungarian Restaurant,Playground,Restaurant,Eastern European Restaurant,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop
1,3,Devin,Café,Restaurant,Castle,Bistro,Park,Cafeteria,River,Lighthouse,Eastern European Restaurant,Gastropub
2,2,Devinska Nova Ves,Beer Garden,Café,Bar,Gym,BBQ Joint,Baseball Field,Supermarket,Pizza Place,Yoga Studio,Dog Run
3,3,Dubravka,Restaurant,Dessert Shop,Ice Cream Shop,Italian Restaurant,Gastropub,Supermarket,Playground,Eastern European Restaurant,Czech Restaurant,Diner
4,1,Jarovce,Grocery Store,Eastern European Restaurant,Pizza Place,Gastropub,Soccer Field,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant


In [42]:
# Joining lat lon values
bratislava_venues_merged = bratislava_venues_sorted.merge(bratislava[['lat', 'lon']],how = "inner", left_on = "cadastral_name", right_on = bratislava["name"])
bratislava_venues_merged.head()

,Cluster Labels,cadastral_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,lat,lon
0,3,Cunovo,Hungarian Restaurant,Playground,Restaurant,Eastern European Restaurant,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop,48.029458,17.198465
1,3,Devin,Café,Restaurant,Castle,Bistro,Park,Cafeteria,River,Lighthouse,Eastern European Restaurant,Gastropub,48.174644,16.983647
2,2,Devinska Nova Ves,Beer Garden,Café,Bar,Gym,BBQ Joint,Baseball Field,Supermarket,Pizza Place,Yoga Studio,Dog Run,48.209720,16.975013
3,3,Dubravka,Restaurant,Dessert Shop,Ice Cream Shop,Italian Restaurant,Gastropub,Supermarket,Playground,Eastern European Restaurant,Czech Restaurant,Diner,48.186997,17.038855
4,1,Jarovce,Grocery Store,Eastern European Restaurant,Pizza Place,Gastropub,Soccer Field,Diner,Coffee Shop,Convenience Store,Cupcake Shop,Czech Restaurant,48.065125,17.113509


In [38]:
## my local geojason of bratislava cadastral districts
with open('cadastral_districts_ba.geojson', 'r', encoding='utf-8') as f:
    bratislava_geo = f.read()

In [39]:
map_clusters_ba_ = folium.Map(location = [location_sk.latitude, location_sk.longitude], zoom_start=10)

map_clusters_ba_.choropleth(
    geo_data = bratislava_geo,
    data = bratislava_venues_merged,
    columns = ['cadastral_name','Cluster Labels'],
    key_on='feature.properties.NM5',
    fill_color='PuBuGn',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name = 'Cluster'
)

markers_colors = []
for lat, lon, poi, cluster in zip(bratislava_venues_merged['lat'], bratislava_venues_merged['lon'], bratislava_venues_merged['cadastral_name'], bratislava_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color= rainbow[cluster-1],# 'PuBuGn',
        fill=True,
        fill_color= rainbow[cluster-1],# 'PuBuGn'
    ).add_to(map_clusters_ba_)
    
map_clusters_ba_

In [51]:
bratislava_venues_merged.loc[bratislava_venues_merged['Cluster Labels'] == 4 , bratislava_venues_merged.columns[[1] + list(range(2, bratislava_venues_merged.shape[1]))]]

,cadastral_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,lat,lon
17,Vinohrady,Campground,Buffet,Ski Chairlift,Mountain,Dog Run,Convenience Store,Cupcake Shop,Czech Restaurant,Dessert Shop,Diner,48.19369,17.097417
